# Custom Types

## Defining data types

We can define types (i.e. data structures) ourselves using the `struct` keyword.

It is a convention that type names are capitalized and [camel cased](https://en.wikipedia.org/wiki/Camel_case).

(Note that types can not be redefined - you have to restart your Julia session to change a type definiton.)

In [1]:
struct MyType end

To create an object of type `MyType` we have to call a [constructor](https://docs.julialang.org/en/latest/manual/constructors/). Loosely speaking, a constructor is a function that create new objects.

Julia automatically creates a trivial constructors for us, which has the same name as the type.

In [2]:
methods(MyType)

# 1 method for type constructor:
[1] MyType() in Main at In[1]:1

In [3]:
m = MyType()

MyType()

In [4]:
typeof(m)

MyType

In [5]:
m isa MyType

true

In [ ]:
import PackageA: MyType
import PackageB: MyType

In [ ]:
PackageA.MyType

In [ ]:
const MyA = PackageA.MyType
const MyB = PackageB.MyType

Since no data is contained in our `MyType`  - it is a so-called *singleton type* - we can basically only use it for dispatch.

Most of the time, we'll want a self-defined type to hold some data. For this, we need *fields*.

In [6]:
struct A
    x::Int64
end

In [7]:
A()

LoadError: MethodError: no method matching A()
Closest candidates are:
  A(!Matched::Int64) at In[6]:2
  A(!Matched::Any) at In[6]:2

The default constructor always expects values for all fields.

In [8]:
A(3)

A(3)

In [9]:
a = A(3)

A(3)

In [10]:
# a.<TAB>
a.x

3

In [11]:
a.x

3

Note that types defined with `struct` are **immutable**, that is the values of it's fields cannot be changed.

In [12]:
a.x = 2

LoadError: setfield! immutable struct of type A cannot be changed

In [13]:
mutable struct B
    x::Int64
end

In [14]:
b = B(3)

B(3)

In [15]:
b.x

3

In [16]:
b.x = 4

4

In [17]:
b.x

4

Note, however, that **immutability is not recursive**.

In [18]:
struct C
    x::Vector{Int64}
end

In [19]:
c = C([1, 2, 3])

C([1, 2, 3])

In [20]:
c.x

3-element Array{Int64,1}:
 1
 2
 3

In [21]:
c.x = [3,4,5]

LoadError: setfield! immutable struct of type C cannot be changed

In [22]:
c.x[1] = 3

3

In [23]:
c.x

3-element Array{Int64,1}:
 3
 2
 3

In [ ]:
c.x .= [3,4,5] # dot to perform the assignment element-wise

In [ ]:
f(M::Matrix) = M.^2

In [24]:
f(x::Float64) = x^2

f (generic function with 1 method)

In [26]:
f.([1.0,2.0,3.0])

3-element Array{Float64,1}:
 1.0
 4.0
 9.0

Abstract types are just as easy to define using the keyword `abstract type`.

In [27]:
abstract type MyAbstractType end

Since abstract types don't have fields, they only (informally) define interfaces and can be used for dispatch.

In [28]:
struct MyConcreteType <: MyAbstractType # subtype
    somefield::String
end

In [29]:
c = MyConcreteType("test")

MyConcreteType("test")

In [30]:
c isa MyAbstractType

true

In [31]:
supertype(MyConcreteType)

MyAbstractType

In [32]:
subtypes(MyAbstractType)

1-element Array{Any,1}:
 MyConcreteType

## Example: Diagonal Matrix

In [33]:
struct DiagMat
    diag::Vector{Float64}
end

In [34]:
DiagMat([1.2,4.3,5.0])

DiagMat([1.2, 4.3, 5.0])

### Arithmetic

In [35]:
import Base: +, -, *, /

+(Da::DiagMat, Db::DiagMat) = DiagMat(Da.diag + Db.diag)
-(Da::DiagMat, Db::DiagMat) = DiagMat(Da.diag - Db.diag)
*(Da::DiagMat, Db::DiagMat) = DiagMat(Da.diag .* Db.diag)
/(Da::DiagMat, Db::DiagMat) = DiagMat(Da.diag ./ Db.diag)

/ (generic function with 119 methods)

In [36]:
D1 = DiagMat([1,2,3])
D2 = DiagMat([2.4,1.9,5.7])

DiagMat([2.4, 1.9, 5.7])

In [37]:
D1 + D2

DiagMat([3.4, 3.9, 8.7])

In [38]:
D1 - D2

DiagMat([-1.4, 0.10000000000000009, -2.7])

In [39]:
D1 * D2

DiagMat([2.4, 3.8, 17.1])

In [40]:
D1 / D2

DiagMat([0.4166666666666667, 1.0526315789473684, 0.5263157894736842])

In [41]:
DiagMat * 2

LoadError: MethodError: no method matching *(::Type{DiagMat}, ::Int64)
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:538
  *(!Matched::Complex{Bool}, ::Real) at complex.jl:309
  *(!Matched::Missing, ::Number) at missing.jl:115
  ...

Arithmetics involving other types:

In [42]:
# Number
*(x::Number, D::DiagMat) = DiagMat(x * D.diag)
*(D::DiagMat, x::Number) = DiagMat(D.diag * x)
/(D::DiagMat, x::Number) = DiagMat(D.diag / x)

# Vector
*(D::DiagMat, V::AbstractVector) = D.diag .* V

* (generic function with 371 methods)

In [43]:
D1 * 2

DiagMat([2.0, 4.0, 6.0])

In [44]:
D1 * rand(3)

3-element Array{Float64,1}:
 0.5524143739391625
 1.284674110687157
 2.501348948027179

Note that some functions already work for our `DiagonalMat`:

In [45]:
sum([D1, D2])

DiagMat([3.4, 3.9, 8.7])

In [46]:
DiagMat([1,2,3])

DiagMat([1.0, 2.0, 3.0])

### Parameterization

In [47]:
DiagMat([1,2,3]) # implicit conversion to Vector{Float64}

DiagMat([1.0, 2.0, 3.0])

In [48]:
DiagMat([1+3im, 4-2im, im])

LoadError: InexactError: Float64(1 + 3im)

In [49]:
DiagMat(["Why", "not", "support", "strings?"])

LoadError: MethodError: Cannot `convert` an object of type String to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number at number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

We can easily relax our type definition to allow all sorts of internal value types.

In [50]:
struct DiagMatParam{T, V<:AbstractVector{T}}
    diag::V
end





# copied from above
import Base: +, -, *, /
+(Da::DiagMatParam, Db::DiagMatParam) = DiagMatParam(Da.diag + Db.diag)
-(Da::DiagMatParam, Db::DiagMatParam) = DiagMatParam(Da.diag - Db.diag)
*(Da::DiagMatParam, Db::DiagMatParam) = DiagMatParam(Da.diag .* Db.diag)
/(Da::DiagMatParam, Db::DiagMatParam) = DiagMatParam(Da.diag ./ Db.diag)
# Number
*(x::Number, D::DiagMatParam) = DiagMatParam(x * D.diag)
*(D::DiagMatParam, x::Number) = DiagMatParam(D.diag * x)
/(D::DiagMatParam, x::Number) = DiagMatParam(D.diag / x)
# Vector
*(D::DiagMatParam, V::AbstractVector) = D.diag .* V

* (generic function with 375 methods)

In [51]:
DiagMatParam([1+3im, 4-2im, im])

DiagMatParam{Complex{Int64},Array{Complex{Int64},1}}(Complex{Int64}[1 + 3im, 4 - 2im, 0 + 1im])

In [52]:
DiagMatParam(["This ", "just "]) * DiagMatParam(["should", "work!"])

DiagMatParam{String,Array{String,1}}(["This should", "just work!"])

### `AbstractArray` interface

Let's integrate our diagonal matrix into Julia's type hierarchy by subtyping `AbstractMatrix`. Of course, we should then also implement the [`AbstractArray` interface](https://docs.julialang.org/en/v1/manual/interfaces/#man-interface-array)!

In [81]:
struct DiagonalMatrix{T, V<:AbstractVector{T}} <: AbstractMatrix{T}
    diag::V
end

In [54]:
zero(Float64)

0.0

In [55]:
zero(Int64)

0

In [56]:
zero(String)

LoadError: MethodError: no method matching zero(::Type{String})
Closest candidates are:
  zero(!Matched::Type{Dates.DateTime}) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Dates/src/types.jl:404
  zero(!Matched::Type{Dates.Date}) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Dates/src/types.jl:405
  zero(!Matched::Type{Pkg.Resolve.FieldValue}) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Pkg/src/Resolve/fieldvalues.jl:38
  ...

In [82]:
# implement AbstractArray interface
Base.size(D::DiagonalMatrix) = (length(D.diag), length(D.diag))

function Base.getindex(D::DiagonalMatrix{T,V}, i::Int, j::Int) where {T,V}
    if i == j
        r = D.diag[i]
    else
        r = zero(T)
    end
    r
end

function setindex!(D::DiagonalMatrix, v, i::Int, j::Int)
    if i == j
        D.diag[i] = v
    else
        throw(ArgumentError("cannot set off-diagonal entry ($i, $j)"))
    end
    return v
end

setindex! (generic function with 1 method)

In [58]:
D = DiagonalMatrix([1,2,3])

3×3 DiagonalMatrix{Int64,Array{Int64,1}}:
 1  0  0
 0  2  0
 0  0  3

Note how it's automagically pretty printed!

In [59]:
D * D

3×3 Array{Int64,2}:
 1  0  0
 0  4  0
 0  0  9

In [60]:
D + D

3×3 Array{Int64,2}:
 2  0  0
 0  4  0
 0  0  6

In [61]:
D - D

3×3 Array{Int64,2}:
 0  0  0
 0  0  0
 0  0  0

In [62]:
D / D

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

Basic arithmetics **just works!** What about broadcasting and more complicated functions?

In [63]:
sin.(D)

3×3 Array{Float64,2}:
 0.841471  0.0       0.0
 0.0       0.909297  0.0
 0.0       0.0       0.14112

In [64]:
sum([D, D, D])

3×3 Array{Int64,2}:
 3  0  0
 0  6  0
 0  0  9

In [65]:
using LinearAlgebra
eigen(D)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
3-element Array{Float64,1}:
 1.0
 2.0
 3.0
vectors:
3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

It is still advantageous to define fast versions that utilize the special diagonal structure:

In [77]:
@which D + D

+(Da::DiagonalMatrix, Db::DiagonalMatrix) in Main at In[76]:1

In [86]:
+(Da::DiagonalMatrix, Db::DiagonalMatrix) = DiagonalMatrix(Da.diag .+ Db.diag)
*(Da::DiagonalMatrix, Db::DiagonalMatrix) = DiagonalMatrix(Da.diag .* Db.diag)
*(x::Number, D::DiagonalMatrix) = DiagonalMatrix(x .* D.diag)

* (generic function with 377 methods)

In [79]:
@which D + D

+(Da::DiagonalMatrix, Db::DiagonalMatrix) in Main at In[78]:1

An important thing to note is that **user defined types are just as good as built-in types**!

There is nothing special about built-in types. In fact, [they are implemented in precisely the same way](https://github.com/JuliaLang/julia/blob/master/stdlib/LinearAlgebra/src/diagonal.jl#L5)!

Let us quickly confirm that our `DiagonalMatrix` type does not come with any performance overhead by benchmarking it in a simple function.

# Benchmarking with `BenchmarkTools.jl`

In [66]:
using BenchmarkTools

In [67]:
operation(x) = x + 2*x

operation (generic function with 1 method)

In [68]:
x = rand(2,2)
@time operation.(x)

  0.087627 seconds (187.21 k allocations: 9.423 MiB)


2×2 Array{Float64,2}:
 2.73677  0.358768
 2.05606  2.94564

In [69]:
function f()
    x = rand(2,2)
    @time operation.(x)
end

f (generic function with 2 methods)

In [70]:
f()

  0.000000 seconds (1 allocation: 112 bytes)


2×2 Array{Float64,2}:
 0.91324  0.271024
 2.27965  0.436537

We should wrap benchmarks into functions!

Fortunately, there are tools that do this for us. In addition, they also collect some statistics by running the benchmark multiple times.

In [71]:
@benchmark operation.(x)

BenchmarkTools.Trial: 
  memory estimate:  176 bytes
  allocs estimate:  4
  --------------
  minimum time:     284.779 ns (0.00% GC)
  median time:      369.904 ns (0.00% GC)
  mean time:        398.406 ns (2.99% GC)
  maximum time:     16.238 μs (95.24% GC)
  --------------
  samples:          10000
  evals/sample:     272

Typically we don't need all this information. Just use `@btime` instead of `@time`!

In [72]:
@btime operation.(x);

  288.431 ns (4 allocations: 176 bytes)


However, we still have to take some care to avoid accessing global variables.

In [73]:
a = 1.23

1.23

In [74]:
"The values of a is $(a)"

"The values of a is 1.23"

In [75]:
@btime operation.($x); # interpolate the value of x into the expression to avoid overhead of globals

  42.607 ns (1 allocation: 112 bytes)


More information: [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/doc/manual.md).

Finally, we can check the performance of our custom volume type.

In [89]:
using LinearAlgebra
x = rand(100);
Djl = Diagonal(x)
D = DiagonalMatrix(x)
@btime operation(Djl);
@btime operation(D);

  171.371 ns (3 allocations: 1.77 KiB)
  178.715 ns (3 allocations: 1.77 KiB)


# Core messages of this Notebook

* **User defined types are as good as built-in types.**
* There are `mutable struct`s and immutable `struct`s and immutability is not recursive.
* We can easily **extend `Base` functions** for our types to implement arithmetics and such.
* **Subtyping an existing interface** can give lots of functionality for free.
* We should always benchmark our code with **BenchmarkTools.jl's @btime and @benchmark**.